# Full Text Analysis with BERT

### Requirements

In [3]:
# Possible Installations
# !pip install sentence_transformers
# # !pip install transformers
# !pip install spacy

In [4]:
import collections
import re
import unicodedata

import nltk
# from collections.abc import Mapping
# from collections.abc import Mapping

import spacy
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import TreebankWordTokenizer, sent_tokenize, word_tokenize
from nltk.tokenize.toktok import ToktokTokenizer

nltk.download('punkt')

import io
import itertools
import os
import shutil
import string

import contractions
import gensim.downloader as api
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import torch
import torchvision
from gensim.models import Word2Vec
from pdfminer3.converter import PDFPageAggregator, TextConverter
from pdfminer3.layout import LAParams, LTTextBox
from pdfminer3.pdfinterp import PDFPageInterpreter, PDFResourceManager
from pdfminer3.pdfpage import PDFPage
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
from spacy.lang.en import English
from transformers import pipeline

import sys
sys.path.append(r"..")
from nlp_functions import (classifier, remove_colons, remove_digits, remove_n,
                           remove_redundant_whitespaces,
                           remove_strange_characters, remove_stripes,
                           text_loader)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\daveb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Preprocessing

In [5]:
def preprocessing(text):
    # mapping = str.maketrans('', '', string.digits)
    # text = text.translate(mapping)

    text = remove_strange_characters(text)
    text = re.sub(r'\d+', '', text)


    text = remove_n(text)

    text = remove_colons(text)

    text = text.replace(r" .",".")
    text = text.replace(r"..",".")
    text = text.replace(r"...",".")

    text = remove_stripes(text)

    text = remove_redundant_whitespaces(text)
    return text

In [7]:
nltk.download('omw-1.4')
nltk.download('wordnet')

def lemmatize_words(text):

    lemmatizer = WordNetLemmatizer()
    text = word_tokenize(text)

    lemma_list = []

    for word in text:
        lemma_word = lemmatizer.lemmatize(word)
        lemma_list.append(lemma_word)

    lemma_text = ' '.join(lemma_list)


    return lemma_text



[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\daveb\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\daveb\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
def remove_small_tokens(prep_text):
    prep_text_before = prep_text
    print(f"Text before cleaning:  {len(prep_text_before)}")
    print(type(prep_text_before))
    # prep_text_before = [prep_text_before]
    prep_text_before = word_tokenize(prep_text_before)
    # print(prep_text_before)

    for word in prep_text_before:
        if len(word) <= 3:
            prep_text_before.remove(word)
    print(f"Text after cleaning:  {len(prep_text_before)}")
    prep_text_list = ' '.join(prep_text_before)
    print(type(prep_text_list))


    return prep_text_list


In [9]:
# lemma_text_cleaned = remove_small_tokens(lemma_text)
# stem_p_text_cleaned = remove_small_tokens(porter_stem_text)
# snowball_stem_text_cleand = remove_small_tokens(snowball_stem_text)

## Part 1: Bag of Words

In [10]:
def get_top_n_words(corpus, n=None):
    corpus=[corpus]

    vec = CountVectorizer(stop_words = 'english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    top_n_words_df = pd.DataFrame(words_freq, columns =['Word', 'Count'])

    return top_n_words_df.loc[:(n-1)]



# Part 2: TF_IDF

In [11]:
# !pip3 install scikit-learn
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer


In [12]:
def get_tf_idf(text, n=None):
    text = re.findall(r'(?:\d[.]|[^.])*(?:[.]|$)', text)
    tfIdfVectorizer=TfidfVectorizer(use_idf=True)
    tfIdf = tfIdfVectorizer.fit_transform(text)
    df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
    df = df.sort_values('TF-IDF', ascending=False)
    return (df[:(n)])

# Auto Script

In [16]:
def pdf_loader(company_name):
    company_name = company_name[:-4]
    source = r'..\Data\Nachhaltigkeitsberichte\Alle'
    path = rf"{source}\{company_name}.pdf"

    text = text_loader(path)

    return text, company_name, path, source

In [19]:
list_of_companies = os.listdir(
    r"..\Data\Nachhaltigkeitsberichte\Alle")
    
for company in list_of_companies:
    ## BOW / TF-IDF
    print(f"We process now {company}")

    text_1, company_name, path, source = pdf_loader(company)
    prep_text = preprocessing(text_1)
    lemma_words = lemmatize_words(prep_text)
    text_cleaned = remove_small_tokens(lemma_words)
    top_n_words = get_top_n_words(text_cleaned, 40)
    top_n_words = list(top_n_words.itertuples(index=False, name=None))
    top_n_words
    tf_idf = get_tf_idf(text_cleaned, 40)
    tf_idf = tf_idf.reset_index()
    tf_idf = tf_idf.rename(columns={"index": "Word"})
    idf_list = list(tf_idf.itertuples(index=False, name=None))
    idf_list
    data = [(company_name, idf_list, top_n_words)]
    df1 = pd.DataFrame(data, columns = ["PDF Name", "TF-IDF", "Top N Words"])
    # print(df1)
    company_name_new = f"{df1['Top N Words'][0][0][0]}_{df1['Top N Words'][0][1][0]}_{df1['Top N Words'][0][2][0]}"
    print(company_name_new)
    df1["Company Name"] = company_name_new
    df1.to_csv(fr"..\Data\Resultate\Testfolder\TF-IDF 40\{company_name_new}_bow_tf_ifd.csv")
    print("SAFED")

We process now 4f303cec-a12d-480b-accb-7b56f706f60e_axa-ri2020-en-accessible.pdf
Text before cleaning:  104806
<class 'str'>
Text after cleaning:  12738
<class 'str'>
axa_health_customer
SAFED
We process now 4f391131-ad12-ab53-7265-5e6c88840627.pdf


c:\Users\daveb\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Text before cleaning:  2458
<class 'str'>
Text after cleaning:  265
<class 'str'>
global_safety_training
SAFED
We process now 5ZmOsI2P3oe0plCvOThrCySgcDcKCXqj.pdf


c:\Users\daveb\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Text before cleaning:  130968
<class 'str'>
Text after cleaning:  13055
<class 'str'>


...

c:\Users\daveb\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


die_würth_der
SAFED
We process now Zegna-responsibility.pdf
Text before cleaning:  48695
<class 'str'>
Text after cleaning:  5471
<class 'str'>
group_zegna_supplier
SAFED
We process now 5df03cc1-f2ec-65ed-9c3d-da3c343395a5.pdf


c:\Users\daveb\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Text before cleaning:  2334
<class 'str'>
Text after cleaning:  256
<class 'str'>
global_training_qshe
SAFED


c:\Users\daveb\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
